# Load data

In [1]:
!pwd

/home/jovyan/work/Documents/hengchang/sdhadoop/hadoop-client


In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2019-02-26 08:20:01--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://120.52.51.18/files.grouplens.org/datasets/movielens/ml-100k.zip [following]
--2019-02-26 08:20:01--  http://120.52.51.18/files.grouplens.org/datasets/movielens/ml-100k.zip
Connecting to 120.52.51.18:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  2.95MB/s    in 1.6s    

2019-02-26 08:20:03 (2.95 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [3]:
!ls

0_spark_trial_python2.ipynb	3_spark_trial_more_stuff.py  ml-100k.zip
1_spark_trial_python3.ipynb	3_spark_trial_text.ipynb     README.txt
2_neo4j_py2neo.ipynb		docker
3_spark_trial_more_stuff.ipynb	ml-100k


In [4]:
!unzip ml-100k.zip
!rm ml-100k.zip
!cd ml-100k
!rm !(u.user)

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
replace ml-100k/u.user? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
/bin/sh: 1: Syntax error: "(" unexpected


In [5]:
!head ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [14]:
# run as root in iterm
# !hadoop fs -mkdir -p /user/root/ml-100k
# !hadoop fs -put ml-100k/u.user hdfs:///user/root/ml-100k
# !hadoop fs -ls /user/root/ml-100k

sudo: hadoop: command not found
sudo: hadoop: command not found
sudo: hadoop: command not found


In [6]:
import findspark
findspark.init()
import pyspark
import random

In [7]:
conftfos = pyspark.SparkConf().setAll([('spark.executor.cores','1')])

In [8]:
sc = pyspark.SparkContext(appName="3_more_stuff", conf=conftfos)

In [9]:
userRDD = sc.textFile("hdfs:///user/root/ml-100k/u.user")

In [10]:
userRDD.count()

943

In [11]:
# # https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial
# from pyspark import SparkContext, SparkConf

# conf = SparkConf().setAppName('MyFirstStandaloneApp')
# sc = SparkContext(conf=conf)

# userRDD = sc.textFile("hdfs:///user/root/ml-100k/u.user")

def parse_N_calculate_age(data):
             userid,age,gender,occupation,zip = data.split("|")
             return  userid, age_group(int(age)),gender,occupation,zip,int(age)

def  age_group(age):
        if age < 10 :
           return '0-10'
        elif age < 20:
           return '10-20'
        elif age < 30:
           return '20-30'
        elif age < 40:
           return '30-40'
        elif age < 50:
           return '40-50'
        elif age < 60:
           return '50-60'
        elif age < 70:
           return '60-70'
        elif age < 80:
           return '70-80'
        else :
           return '80+'

data_with_age_bucket = userRDD.map(parse_N_calculate_age)

RDD_20_30 = data_with_age_bucket.filter(lambda line : '20-30' in line)

freq = RDD_20_30.map(lambda line : line[3]).countByValue()

print ("total user count is ",userRDD.count())

print ("total movie users profession wise ",dict(freq))

Under_age = sc.accumulator(0)
Over_age = sc.accumulator(0)

def outliers(data):
    global Over_age, Under_age
    age_grp = data[1]
    if(age_grp == "70-80"):
        Over_age +=1
    if(age_grp == "0-10"):
        Under_age +=1
    return data

df = data_with_age_bucket.map(outliers).collect()

print ("under age users of the movie are ",Under_age)
print ("over age users of the movie are ",Over_age)

total user count is  943
total movie users profession wise  {'entertainment': 8, 'artist': 12, 'engineer': 23, 'administrator': 19, 'executive': 7, 'educator': 12, 'doctor': 2, 'homemaker': 3, 'lawyer': 4, 'scientist': 8, 'technician': 12, 'other': 38, 'librarian': 11, 'writer': 14, 'student': 116, 'programmer': 30, 'salesman': 2, 'none': 2, 'healthcare': 4, 'marketing': 5}
under age users of the movie are  1
over age users of the movie are  4


In [12]:
sc.stop()